### <font color=lightseagreen>Research questions:</font>

__1. Are there a best and a worst day of the months to invest in (buy stocks from) <font color=deeppink>the MSCI world index</font> and <font color=deeppink>S&P 500</font>?__

__2. If question 1 is proven to be true, will the following factors have an effect on them:__
> - __`time`__: do the best and the worst day move over the years or they stay constant. 
> - __`federal holidays of the year`__: for this project we will focus first on public holidays of the US
> - how about __`corona crisis`__?

__3. If they indeed move, by how much have they moved?__

__4. Will we see similar effect based on daytime?__

__5. As rumour has it, May and/or October might be the sour months for stock investment. Is this really true?__

__6. Will blue-chip stocks like <font color=deeppink>Apple, Microsoft, SAP, Unilever, etc...</font> follow the same trend, wrt worst and best day for stock purchases?__

### <font color=lightseagreen>Chosen datasets:</font>

For the research questions above, the relevant data would logically be, and limited to (for scope setting), the following:
1. Financial data of: <font color=lightseagreen>(by using yfinance API)</font>
    1. MSCI world index
    2. S&P 500
    3. Apple
    4. Microsoft
2. Public holidays of the US <font color=lightseagreen>(via web-scraping)</font>

__Notes:__
> As I am more interested in effect over a long period of time, the lowest granularity level of my data is __`per day`__.

### The most important things I have learned:
1. How to work with yfinance API/library via python
2. Various data granularity levels that can be extracted via yfinance and the limitation associated with them
    1. 1m data is only available for last 7 days
    2. data interval <1d for the last 60 days
3. Regarding web-scraping, important to check the robots.text before scraping
> wed-scraping from `https://www.nationaldayarchives.com/day-category/official-holidays-us/` --> `forbidden access`
4. How to work with python module Datetime
5. Some learning with text processing

### <font color=lightseagreen>Codes:</font>

### <font color=lightseagreen>Downloading financial data</font>

In [56]:
import yfinance as yf
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import openpyxl
from openpyxl.workbook import Workbook
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [57]:
tickers = ['MSCI', '^GSPC', 'AAPL', 'MSFT', 'GOOG']
concat = pd.DataFrame()
for ticker in tickers:
    dataset = yf.download(ticker, period='max')
    dataset['Company'] = ticker
    concat = pd.concat([concat, dataset], ignore_index=False)

concat.reset_index(inplace=True)

# date_col = pd.to_datetime(concat['Date'])
# concat.drop(columns=['Date'], inplace=True)
# final_df = pd.concat([concat, date_col], axis=1, ignore_index=False)
# final_df.reset_index(drop=True, inplace=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [58]:
concat.Company.value_counts()

^GSPC    23453
AAPL     10191
MSFT      8865
GOOG      4213
MSCI      3396
Name: Company, dtype: int64

### <font color=lightseagreen>Webscraping US federal holidays</font>

In [59]:
url = 'https://en.wikipedia.org/wiki/Federal_holidays_in_the_United_States'
html = requests.get(url)
soup = BeautifulSoup(html.text, 'lxml')

In [60]:
tables = soup.find_all('table')
fed_holidays_table = tables[1]   # "list of federal holidays" is located at position 1 on tables sequence
fed_holidays_data = fed_holidays_table.find_all('td')

In [61]:
# extracting the first column of table data
dates = [fed_holidays_data[i].text for i in range(len(fed_holidays_data)) if i%3==0] 

# removing the parentheses from each item in the list
new_dates = [date.replace('(', '').replace(')', '') for date in dates]

In [62]:
# this is to separate the fixed holidays and the floating holidays from each other for further processing
fixed = [date.split(' Fixed')[0] for date in new_dates if len(date.split(' ')) == 3]
floating = [date for date in new_dates if len(date.split(' ')) > 3]      

In [63]:
# extracting the lowest and highest year from the stock dataset "concat"
years = concat['Date'].apply(lambda x: x.strftime('%Y'))
min_year = int(min(years))
max_year = int(max(years))
years_range = list(range(min_year, max_year+1, 1))

In [64]:
# dealing with the floating holidays
# an item in the floating list is a string like so 'February 15-27 Floating Monday'
fed_holidays = []
for year in years_range:
    for item in floating:
        split = re.findall('(\w+)', item) # split the string at white space
        min_day = int(split[1])           # extract the first number
        max_day = int(split[2])+1         # extract the second number
        for i in range(min_day, max_day): 
            date = ' '.join([split[0], str(i)])
            date = ', '.join([date, str(year)])
            date = datetime.strptime(date, '%B %d, %Y')  # '%B %d, %Y' format is equivalent to 'January 22, 1984'
            # to check which date in the range is equivalent to the last word in the floating item, e.g. 'Monday'
            if date.strftime("%A")==split[-1]:           # '%A' to extract the day, e.g. 'Monday'
                fed_holidays.append(date)

In [65]:
# dealing with the fixed holidays
for year in years_range:
    for i in fixed:
        date_str = ', '.join([i, str(year)])
        date = datetime.strptime(date_str, '%B %d, %Y')
        fed_holidays.append(date)

In [75]:
new_dates

['January 1 Fixed',
 'January 15–21 Floating Monday',
 'February 15–21 Floating Monday',
 'May 25–31 Floating Monday',
 'July 4 Fixed',
 'September 1–7 Floating Monday',
 'October 8–14 Floating Monday',
 'November 11 Fixed',
 'November 22–28 Floating Thursday',
 'December 25 Fixed']

In [74]:
len(years_range)

95

In [67]:
min_year

1927

In [76]:
fed_holidays

[Timestamp('1927-01-17 00:00:00'),
 Timestamp('1927-02-21 00:00:00'),
 Timestamp('1927-05-30 00:00:00'),
 Timestamp('1927-09-05 00:00:00'),
 Timestamp('1927-10-10 00:00:00'),
 Timestamp('1927-11-24 00:00:00'),
 Timestamp('1928-01-16 00:00:00'),
 Timestamp('1928-02-20 00:00:00'),
 Timestamp('1928-05-28 00:00:00'),
 Timestamp('1928-09-03 00:00:00'),
 Timestamp('1928-10-08 00:00:00'),
 Timestamp('1928-11-22 00:00:00'),
 Timestamp('1929-01-21 00:00:00'),
 Timestamp('1929-02-18 00:00:00'),
 Timestamp('1929-05-27 00:00:00'),
 Timestamp('1929-09-02 00:00:00'),
 Timestamp('1929-10-14 00:00:00'),
 Timestamp('1929-11-28 00:00:00'),
 Timestamp('1930-01-20 00:00:00'),
 Timestamp('1930-02-17 00:00:00'),
 Timestamp('1930-05-26 00:00:00'),
 Timestamp('1930-09-01 00:00:00'),
 Timestamp('1930-10-13 00:00:00'),
 Timestamp('1930-11-27 00:00:00'),
 Timestamp('1931-01-19 00:00:00'),
 Timestamp('1931-02-16 00:00:00'),
 Timestamp('1931-05-25 00:00:00'),
 Timestamp('1931-09-07 00:00:00'),
 Timestamp('1931-10-

In [80]:
type(df.Date[0])

pandas._libs.tslibs.timestamps.Timestamp

In [69]:
# this step is necessary to turn the fed holidays into pandas timestamp dtype. 
# this allows for mapping with the date column in the stock dataset 'concat'
df = pd.DataFrame(fed_holidays, columns=['Date'])
fed_holidays = df['Date'].tolist()

# creating a dict based on fed_holidays list: 
# the keys will be the timestamps from the list itself
# the values will be just the word 'Yes'
new_dict = dict(zip(fed_holidays, ['Yes']*len(fed_holidays)))

In [81]:
# map the new_dict with the column 'Date' in concat to indicate if a date is a fed holiday or not.
# new column 'Holiday' is created to contain only 'Yes', when a date is a holiday and np.nan, when there is no match
concat['Holiday'] = concat['Date'].map(new_dict)

# checking if column 'Holiday' indeed contains 'Yes' and np.nan
concat.Holiday.value_counts()

Yes    494
Name: Holiday, dtype: int64

In [72]:
# writing the dataset concat to an excel file
wb = Workbook()
ws = wb.active
for r in dataframe_to_rows(concat, index=False, header=True):
    ws.append(r)
wb.save('../data-folder/dataset.xlsx')

### <font color=lightseagreen>Challenges:</font>